In [ ]:
import numpy as np
import pandas as pd
#import tensorflow as tf


import contractions
import os
from collections import Counter
from bs4 import BeautifulSoup
# import torch
# from torch.utils.data import DataLoader, Dataset

# from transformers import BertForSequenceClassification, BertTokenizer
# import torch

# from torch.utils.data.sampler import SubsetRandomSampler
# import torch.nn as nn
# from torch.utils.data import DataLoader, TensorDataset
# cuda = torch.device('cuda')


import nltk
# nltk.download('wordnet')
# nltk.download('omw-1.4')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

import nlpaug
import nlpaug.augmenter.word as naw

%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
!pip install yfinance
import yfinance as yf

from pandas.tseries.offsets import DateOffset
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
path = '../data/Transcripts'
df = pd.DataFrame()
companies = {'AMZN':'Amazon',
 'AAPL':'Apple',
 'INTC':'Intel',
 'MU':'Micron',
 'GOOGL': 'Alphabet',
 'MSFT': 'Microsoft',
 'AMD':'AMD',
 'ASML':'ASML',
 'NVDA':'NVIDIA',
 'CSCO':'Cisco'}

In [ ]:
df = []
tf = 0
totalsummary = ''
totalqna = ''
for comp in os.listdir(path):
    if comp!='.DS_Store':
        company = comp
        files = os.listdir(path+'/'+company)
        for j in files:
            allinfo = []
            with open(path+'/'+company+'/'+j) as f:
                lines = f.readlines()
                summary = []
                qna = []
                i = 0
                while i<len(lines):
                    if lines[i]=='\n':
                        i+=1
                    elif lines[i][:3]=='===':
                        i+=1
                    elif 'operator' in lines[i].lower():
                        i+=3
                    elif lines[i][:3] == '---':
                        i+=1
                    elif 'presentation' in lines[i].lower():
                        i+=2
#                         print(j)
                        while i<len(lines):
#                             print(lines[i])
                            if '===' in lines[i]:
                                break
                            if 'Questions and Answers' in lines[i]:
                                break
                            elif lines[i]=='\n':
                                i+=1
                                continue
                            elif 'operator' in lines[i].lower():
                                i+=3
                            elif '---' in lines[i]:
                                i+=1
                            summary.append(lines[i].rstrip())
                            i+=1
                    
                    elif '[' in lines[i] and companies[company].lower() in lines[i].lower():
                        i+=2
                        while lines[i][:3]!='---':
                            if lines[i]=='\n':
                                i+=1
                                continue
                            summary.append(lines[i].rstrip())
                            i+=1
                    
                    elif 'Questions and Answers' in lines[i]:
                        i+=1
                        while i<len(lines):
                            if lines[i]=='\n':
                                i+=1
                            elif 'Operator' in lines[i]:
                                i+=2
                            elif lines[i][:3] == '---':
                                i+=1
                            elif '[' in lines[i]:
                                i+=2
                            else:
                                qna.append(lines[i].rstrip())
                                i+=1
                    
                    elif '[' in lines[i] and companies[company].lower() not in lines[i].lower():
                        while i<len(lines):
                            if lines[i]=='\n':
                                i+=1
                            elif 'Operator' in lines[i]:
                                i+=3
                            elif lines[i][:3] == '---':
                                i+=1
                            elif '[' in lines[i]:
                                i+=2
                            else:
                                qna.append(lines[i].rstrip())
                                i+=1
                    else:
                        i+=1
                
                if lines[0][0] == 'Q':
                    quarter = lines[0].split(' ')[0]
                    year = lines[0].split(' ')[1]
                else:
                    quarter = lines[1].split(' ')[0]
                    year = lines[1].split(' ')[1]
                cleansummary = []
#                 print(j)
                for i in range(len(summary)):
                    if '--' in summary[i]:
                        i+=1
                    elif 'Operator' in summary[i]:
                        i+=3
                    elif '[' in summary[i]:
#                         print(i, summary[i])
                        i+=1
                    else:
                        cleansummary.append(summary[i])
                summary_para = ' '.join(cleansummary).rstrip()
                qna_para = ' '.join(qna).rstrip()
                totalsummary += summary_para
                totalqna = qna_para
                allinfo.append(quarter)
                allinfo.append(year)
                allinfo.append(companies[company])
                allinfo.append(summary_para)
                allinfo.append(qna_para)
                df.append(allinfo)

In [ ]:
earnings_transcript_df = pd.DataFrame(df)
earnings_transcript_df.columns =['Quarter','Year','Company','Summary','Q&A']

In [ ]:
earnings_transcript_df

In [ ]:
words = Counter(totalsummary.split(' '))
words += Counter(totalqna.split(' '))

In [ ]:
from readability import Readability
r = Readability(earnings_transcript_df.iloc[0,3])
r.flesch_kincaid()
print(r.flesch())
print(r.gunning_fog())

In [ ]:
def getAvgTxtLen(df):
    avg_length_summary_after_preprocessing = str(df['Summary'].str.len().mean()) 
    avg_length_qna_after_preprocessing = str(df['Q&A'].str.len().mean()) 
    print(avg_length_summary_after_preprocessing)
    print(avg_length_qna_after_preprocessing)

In [ ]:
def strip_html_tags(text): 
    soup = BeautifulSoup(text, "html.parser") 
    stripped_text = soup.get_text(separator=" ") 
    return stripped_text 
    
# Make lowercase
earnings_transcript_df["Summary"] = earnings_transcript_df["Summary"].str.lower()
earnings_transcript_df["Q&A"] = earnings_transcript_df["Q&A"].str.lower()

# Remove URLS
earnings_transcript_df['Summary']= earnings_transcript_df['Summary'].str.replace('http(s)?:\/\/.*', '', regex=True)
earnings_transcript_df['Q&A']= earnings_transcript_df['Q&A'].str.replace('http(s)?:\/\/.*', '', regex=True)
print("Remove URLS")
getAvgTxtLen(earnings_transcript_df)

# remove HTML tags
earnings_transcript_df['Summary'] = earnings_transcript_df['Summary'].apply(lambda x: strip_html_tags(x))
earnings_transcript_df['Q&A'] = earnings_transcript_df['Q&A'].apply(lambda x: strip_html_tags(x))
print("remove HTML tags")
getAvgTxtLen(earnings_transcript_df)

# remove extra whitespaces
earnings_transcript_df['Summary'] = earnings_transcript_df['Summary'].replace(r'/\s/\s+/g', ' ', regex=True)
earnings_transcript_df['Q&A'] = earnings_transcript_df['Q&A'].replace(r'/\s/\s+/g', ' ', regex=True)
print("remove extra whitespaces")
getAvgTxtLen(earnings_transcript_df)

# remove contractions
earnings_transcript_df['Summary'] = earnings_transcript_df['Summary'].apply(lambda x: ' '.join([contractions.fix(word) for word in x.split()]))
earnings_transcript_df['Q&A'] = earnings_transcript_df['Q&A'].apply(lambda x: ' '.join([contractions.fix(word) for word in x.split()]))
print("remove contractions")
getAvgTxtLen(earnings_transcript_df)

# Remove characters following digits such as 1st, 2nd or 3rd
earnings_transcript_df['Summary'] = earnings_transcript_df['Summary'].str.replace('\b\d+\w+\b', '', regex=True)
earnings_transcript_df['Q&A'] = earnings_transcript_df['Q&A'].str.replace('\b\d+\w+\b', '', regex=True)
print("Remove characters following digits such as 1st, 2nd or 3rd")
getAvgTxtLen(earnings_transcript_df)

# Replace hyphens with space
earnings_transcript_df['Summary'] = earnings_transcript_df['Summary'].replace('/\b-\b/g', ' ', regex=True)
earnings_transcript_df['Q&A'] = earnings_transcript_df['Q&A'].replace('/\b-\b/g', ' ', regex=True)
print("Replace hyphens with space")
getAvgTxtLen(earnings_transcript_df)

# remove non alphabetical characters
earnings_transcript_df['Summary'] = earnings_transcript_df['Summary'].str.replace('[^a-zA-Z0-9 ]', ' ')
earnings_transcript_df['Q&A'] = earnings_transcript_df['Q&A'].str.replace('[^a-zA-Z0-9 ]', ' ')
print("remove non alphabetical characters")
getAvgTxtLen(earnings_transcript_df)


In [ ]:
earnings_transcript_df.iloc[0,2]

In [ ]:
stop_words = stopwords.words('english')
earnings_transcript_df['Summary'] = earnings_transcript_df['Summary'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words) ]))
earnings_transcript_df['Summary'] = earnings_transcript_df['Summary'].astype('string')

earnings_transcript_df['Q&A'] = earnings_transcript_df['Q&A'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words) ]))
earnings_transcript_df['Q&A'] = earnings_transcript_df['Q&A'].astype('string')

print("Stopwords")
getAvgTxtLen(earnings_transcript_df)
# Lemmatize the Dataset
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatize_review(text):
    return ' '.join(lemmatizer.lemmatize(w,pos="v") for w in w_tokenizer.tokenize(text))

earnings_transcript_df['Summary'] = earnings_transcript_df['Summary'].apply(lemmatize_review)
earnings_transcript_df['Summary'] = earnings_transcript_df['Summary'].astype('string')
earnings_transcript_df['Q&A'] = earnings_transcript_df['Q&A'].apply(lemmatize_review)
earnings_transcript_df['Q&A'] = earnings_transcript_df['Q&A'].astype('string')
print("Lemmatize")
getAvgTxtLen(earnings_transcript_df)



In [ ]:
avg_length_summary_after_preprocessing = str(earnings_transcript_df['Summary'].str.len().mean()) 
avg_length_qna_after_preprocessing = str(earnings_transcript_df['Q&A'].str.len().mean()) 
print(avg_length_summary_after_preprocessing)
print(avg_length_qna_after_preprocessing)

In [ ]:
earnings_transcript_df

In [ ]:
# Running augmentations
augdf = earnings_transcript_df.copy()
aug = naw.SynonymAug(aug_src='wordnet', model_path=None, name='Synonym_Aug', aug_min=10, aug_max=60, aug_p=0.3)
for i in range(len(augdf)):
    k = aug.augment(earnings_transcript_df.iloc[i]['Summary'],n=1)
    augdf['Summary'][i] = k[0]
    k = aug.augment(earnings_transcript_df.iloc[i]['Q&A'],n=1)
    augdf['Q&A'][i] = k[0]

earnings_transcript_df = earnings_transcript_df.append(augdf)
earnings_transcript_df = earnings_transcript_df.reset_index().drop(columns = ['index'])

In [ ]:
earnings_transcript_df

In [ ]:
today = pd.to_datetime("today")

# Consolidated Dataframe with sentiments for all companies
finance_df = pd.DataFrame(columns=["Company", "date", "Quarter", "Year", "percent change (between t-2 and t+2)","Sentiment"])

company_tickers = ['NVDA','MU','INTC','GOOGL','CSCO','ASML','AMZN','AMD','AAPL','MSFT']

price_change_perc_threshold_for_sentiment = 3.0

for ticker in company_tickers:
    
    # Get List of dates for Earnings call
    ticker_data = yf.Ticker(ticker)
    ticker_data.earnings_dates.reset_index(inplace=True)
    
    #Format Date column
    ticker_data.earnings_dates["Earnings Date"] = ticker_data.earnings_dates["Earnings Date"].apply(lambda x: x.strftime('%Y-%m-%d'))
    
    # Filter dates within window of our Transcripts
    earnings_dates_df = ticker_data.earnings_dates[(ticker_data.earnings_dates["Earnings Date"]<=today.strftime('%Y-%m-%d')) & (ticker_data.earnings_dates["Earnings Date"]>="2015-12-01")]
    earning_call_dates= earnings_dates_df.iloc[:,0]

    #print(earning_call_dates)
    
    prepared_df = pd.DataFrame(columns=["Company", "date", "Quarter", "Year", "percent change (between t-2 and t+2)","Sentiment"])
    
    historical_data = yf.download(ticker, start="2015-12-01", end="2022-11-21", interval="1d")

    historical_data.reset_index(inplace=True)

    for date in earning_call_dates:
        # Compute Quarters for each date to join finance df with Transcripts DF created in above cells
        month=pd.to_datetime(date).month
        year=pd.to_datetime(date).year
        if month<=3:
            quarter="Q1"
        elif month<=6:
            quarter="Q2"
        elif month<=9:
            quarter="Q3"
        else:
            quarter="Q4"
                
        # Compute % change by looking at T-2 and T+2 window from earnings call date
        # i.e. if Earnings call is 27 Oct 2022 => compute:
        # => (Closing_price['29 Oct'] - Closing_price['27 Oct']) * 100 / (Closing_price['27 Oct'])
        
        index= historical_data.index[historical_data["Date"]==pd.to_datetime(date)]
        tplus2= float(historical_data.loc[index+2,"Adj Close"])
        tminus2 = float(historical_data.loc[index-2,"Adj Close"])
        change = (tplus2 - tminus2)*100/tminus2
        
        # Computer sentiment based on price change percentage threshold
        if abs(change)< price_change_perc_threshold_for_sentiment:
            label="Neutral"
        elif change> price_change_perc_threshold_for_sentiment:
            label="Positive"
        else:
            label="Negative"
        prepared_df = prepared_df.append({"Company":companies[ticker],"date":date,"Quarter":quarter, "Year": year, "percent change (between t-2 and t+2)": change, "Sentiment": label}, ignore_index=True)
    prepared_df.reset_index(drop=True)
    prepared_df.to_csv("..\\data\\OHLC\\"+ticker+".csv", encoding='utf-8')
    finance_df=pd.concat([finance_df, prepared_df], ignore_index=True)
finance_df

In [ ]:
finance_df.Year = finance_df.Year.astype('int')
earnings_transcript_df.Year = earnings_transcript_df.Year.astype('int')

In [ ]:
df_merged = pd.merge(earnings_transcript_df, finance_df, on=['Company', 'Year', 'Quarter'], how='inner')

In [ ]:
df_merged

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing  import LabelEncoder
le = LabelEncoder()
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
import missingno as miss
from collections import Counter
import matplotlib.pyplot as plt
import plotly.express as px
from plotly.offline import iplot
import cufflinks as cf
cf.go_offline()
from tqdm import tqdm

from nltk.corpus import stopwords    
from nltk.tokenize import word_tokenize
from textblob import TextBlob

from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, SpatialDropout1D, MaxPooling1D, Conv1D, Concatenate, Bidirectional, GlobalMaxPool1D, ActivityRegularization, BatchNormalization
from keras.models import Model
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score,confusion_matrix

In [ ]:
train_x_y = df_merged[['Q&A','Sentiment']].sample(frac = 0.80, random_state = 42)
test_x_y = pd.concat([df_merged[['Q&A','Sentiment']], train_x_y]).drop_duplicates(keep=False)
train_x_y["Sentiment"] = le.fit_transform(train_x_y["Sentiment"])
test_x_y["Sentiment"] = le.fit_transform(test_x_y["Sentiment"])

In [ ]:
print(train_x_y.shape)
print(test_x_y.shape)

In [ ]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs


model_args = ClassificationArgs()
model_args.train_batch_size = 2
model_args.gradient_accumulation_steps = 8
model_args.learning_rate = 3e-5
model_args.num_train_epochs = 1

model_bert = ClassificationModel("bert", "bert-base-uncased", num_labels=3, args=model_args, use_cuda=False)

In [ ]:
model_bert.train_model(train_x_y)

In [ ]:
pred_bert, out_bert = model_bert.predict(list(test_x_y['Q&A'].values))

acc_bert = accuracy_score(test_x_y['Sentiment'].to_numpy(), pred_bert)
f1_bert = f1_score(test_x_y['Sentiment'].to_numpy(), pred_bert, average='micro')

print("Accuracy score -->", acc_bert)
print("F1 score -->", f1_bert)

In [ ]:
cm = confusion_matrix(pred_bert, test_x_y['Sentiment'].to_numpy())

In [ ]:
precision = precision_score(pred_bert, test_x_y['Sentiment'].to_numpy(), average='micro')

In [ ]:
recall = recall_score(pred_bert, test_x_y['Sentiment'].to_numpy(), average='micro')

In [ ]:
precision

In [ ]:
recall